## Simulator (single uav)

This notebook serves to get familiar with the simulator

In [1]:
from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

1. **Set the simulation origin in GRA (Global Geodetic Reference) coordinates**  
   * The ENU (East-North-Up) origin is used only for the Gazebo representation. Typically, the ENU origin is taken as $(x=0, y=0)$.

2. **Set UAV base home and path**  
   * **Base home**: The starting UAV position (relative to the simulation origin). It is given in ENU coordinates for easier use.  
   * **Base path**: The UAV trajectory (relative to the uav base home). It is also given in ENU coordinates for easier use.

3. **UAV color**  
   * **Color**: Pick a color for the drone. This color is used for both the drone and its waypoints in Gazebo. For more complex setups with multiple UAVs and GCSs (Ground Control Stations), colors are typically used to distinguish which GCS or provider each UAV belongs to.

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_home = ENUPose(x=5, y=5, z=0, heading=45)
base_path = Plan.create_square_path(side_len=10, alt=5,heading=0)

color =Color.BLUE

## Visualizer

We can choose between three visualization modes:
* Gazebo  
* QGroundControl  
* No visualization  

Below we describe how to configure each of them for completeness.

## Gazebo

In [3]:
gaz_config = ConfigGazebo(origin= enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

gaz_config.add(base_path=base_path,base_home=base_home,color=color)
gaz_config.show()

gaz= Gazebo(gaz_config,gra_origin)

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin= gra_origin)

qgc_config.add(base_path=base_path,base_home=base_home,color=color)
qgc_config.show()

qgc=QGC(qgc_config)

## No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
novis_config.add(base_home=base_home)

novis=NoVisualizer(novis_config)

## Launch Simulation

**Here we configure the simulation:**
* **visualizers**: Select the desired visualizer for the simulation (only one visualizer at a time is currently supported).  
* **terminals**: Processes to display in the terminal. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **suppress_output**: Processes whose output should be suppressed. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **verbose**: Verbosity level (0 = no information, 1 = normal information, 2 = include debugging information).  
* **other arguments**: More advanced options that allow further customization.  


In [10]:
qgc_config.vehicles[0].mission

Mission(traj=[QGCWP(pos=GRA(lat=-35.36328308396274, lon=149.16516902202562, alt=5), color=blue), QGCWP(pos=GRA(lat=-35.3633466047538, lon=149.16509112990263, alt=5), color=blue), QGCWP(pos=GRA(lat=-35.363283083792105, lon=149.16501323798883, alt=5), color=blue), QGCWP(pos=GRA(lat=-35.36321956305101, lon=149.16509113011182, alt=5), color=blue), QGCWP(pos=GRA(lat=-35.36328308396274, lon=149.16516902202562, alt=5), color=blue)], delay=0, n_items=7)

In [6]:
simulator = Simulator(
	visualizers=[novis],
    gcs_names=['simple'],
	gcs_sysids=[[1]],
	missions=[qgc_config.vehicles[0].mission],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

17:51:38 - Oracle ⚪ - INFO - 🙈 Running without visualization.
17:51:38 - Oracle ⚪ - INFO - 🚀 GCS simple launched (PID 368358)
17:51:38 - Oracle ⚪ - INFO - ⏳ Waiting for GCS processes to launch vehicles...
17:51:43 - Oracle ⚪ - INFO - 🔗 Starting Oracle connections to vehicles...
17:51:43 - Oracle ⚪ - INFO - 🔗 UAV logic 1 is connected to Oracle ⚪


## Oracle checking

In [7]:
orac.run()

17:51:43 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 1 vehicles
17:51:43 - Oracle ⚪ - INFO - Monitoring 1 GCS processes
17:53:02 - Oracle ⚪ - INFO - Received message 'DONE' from GCS simple
17:53:02 - Oracle ⚪ - INFO - GCS simple removal complete. Remaining connections: 0
17:53:02 - Oracle ⚪ - INFO - ✅ Main monitoring loop completed - all connections closed
17:53:02 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
